https://blog.openmined.org/upgrade-to-federated-learning-in-10-lines/

https://github.com/OpenMined/PySyft/blob/master/examples/tutorials/Part%206%20-%20Federated%20Learning%20on%20MNIST%20using%20a%20CNN.ipynb

In [3]:
#Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [4]:
#Pysyft and define remote workers alice and bob
import syft as sy #import Pysyft library
hook = sy.TorchHook(torch) #add extra functionalities to support Frderated Learning
bob = sy.VirtualWorker(hook, id="bob") #define 1st worker bob
alice = sy.VirtualWorker(hook, id="alice") #2nd worker  

W0731 14:29:26.089576  6332 secure_random.py:26] Falling back to insecure randomness since the required custom op could not be found for the installed version of TensorFlow. Fix this by compiling custom ops. Missing file was 'C:\Users\Vilas_2\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted/operations/secure_random/secure_random_module_tf_1.14.0.so'
W0731 14:29:26.189540  6332 deprecation_wrapper.py:119] From C:\Users\Vilas_2\Anaconda3\envs\pysyft\lib\site-packages\tf_encrypted\session.py:26: The name tf.Session is deprecated. Please use tf.compat.v1.Session instead.



Define the setting of learning task

In [6]:
class Arguments():
    def __init__(self):
        self.batch_size = 64
        self.test_batch_size = 1000
        self.epochs = 10
        self.lr = 0.01
        self.momentum = 0.5
        self.no_cuda = False
        self.seed = 1
        self.log_interval = 30
        self.save_model = False
        
args = Arguments()

use_cuda = not args.no_cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)

device = torch.device("cuda" if use_cuda else "cpu")

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

Data loading and sending to workers

In [7]:
federated_train_loader = sy.FederatedDataLoader(
    datasets.MNIST('~/.pytorch/MNIST_data/', train=True, download=True,
                   transform=transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ]))
    .federate((bob,alice)), #distribute the dataset acroos all workers. It's a Fderated Dataset
    batch_size=args.batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('~/.pytorch/MNIST_data/', train=False, download=True,
                   transform= transforms.Compose([
                       transforms.ToTensor(),
                       transforms.Normalize((0.1307,), (0.3081,))
                   ])),
    batch_size=args.test_batch_size, shuffle=True, **kwargs)

100.1%

Extracting C:\Users\Vilas_2/.pytorch/MNIST_data/MNIST\raw\train-images-idx3-ubyte.gz


28.4%

113.5%

Extracting C:\Users\Vilas_2/.pytorch/MNIST_data/MNIST\raw\train-labels-idx1-ubyte.gz


100.4%

Extracting C:\Users\Vilas_2/.pytorch/MNIST_data/MNIST\raw\t10k-images-idx3-ubyte.gz


180.4%

Extracting C:\Users\Vilas_2/.pytorch/MNIST_data/MNIST\raw\t10k-labels-idx1-ubyte.gz
Processing...
Done!


NN architecture

In [8]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1,20,5,1)
        self.conv2 = nn.Conv2d(20,50,5,1)
        self.fc1 = nn.Linear(4*4*50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self,x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x,2,2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x,2,2)
        x = x.view(-1, 4*4*50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return F.log_softmax(x, dim=1)

Define the train and test function
For the train fc, because the data batches are distributed across alice and bob, you need to send the model to the right location for each batch. Then, you perform all the operations remotely with the same syntax like doing local pytorch. After done, get back the model updated and the loss to look for improvement

In [9]:
def train(args, model, device, federated_train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data,target) in enumerate(federated_train_loader): #it's a distributed dataset
        model.send(data.location) #send the model to the right location
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        model.get() #get the new model back
        if batch_idx % args.log_interval == 0:
            loss = loss.get() # get the loss back
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * args.batch_size, len(federated_train_loader) * args.batch_size,
                100. * batch_idx / len(federated_train_loader), loss.item()))

In [12]:
#Test fc doesn't change
def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item() #sum up batch loss
            pred = output.argmax(1, keepdim=True) #get the index of the max log-propbability
            correct += pred.eq(target.view_as(pred)).sum().item()
            
        test_loss /= len(test_loader.dataset)
        
        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct / len(test_loader.dataset)))

Training!!

In [13]:
%%time
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=args.lr) #momentum not supported

for epoch in range(1, args.epochs + 1):
    train(args, model, device, federated_train_loader, optimizer, epoch)
    test(args, model, device, test_loader)

if(args.save_model):
    torch.save(model.state_dict(), "mnist_cnn.pt")

Train Epoch: 1 [0/60032 (0%)]	Loss: 2.313616
Train Epoch: 1 [1920/60032 (3%)]	Loss: 2.220160
Train Epoch: 1 [3840/60032 (6%)]	Loss: 2.068735
Train Epoch: 1 [5760/60032 (10%)]	Loss: 1.724446
Train Epoch: 1 [7680/60032 (13%)]	Loss: 1.033717
Train Epoch: 1 [9600/60032 (16%)]	Loss: 0.622254
Train Epoch: 1 [11520/60032 (19%)]	Loss: 0.649745
Train Epoch: 1 [13440/60032 (22%)]	Loss: 0.900343
Train Epoch: 1 [15360/60032 (26%)]	Loss: 0.417282
Train Epoch: 1 [17280/60032 (29%)]	Loss: 0.272216
Train Epoch: 1 [19200/60032 (32%)]	Loss: 0.385671
Train Epoch: 1 [21120/60032 (35%)]	Loss: 0.275365
Train Epoch: 1 [23040/60032 (38%)]	Loss: 0.442085
Train Epoch: 1 [24960/60032 (42%)]	Loss: 0.338076
Train Epoch: 1 [26880/60032 (45%)]	Loss: 0.308777
Train Epoch: 1 [28800/60032 (48%)]	Loss: 0.266436
Train Epoch: 1 [30720/60032 (51%)]	Loss: 0.166424
Train Epoch: 1 [32640/60032 (54%)]	Loss: 0.317791
Train Epoch: 1 [34560/60032 (58%)]	Loss: 0.237456
Train Epoch: 1 [36480/60032 (61%)]	Loss: 0.246858
Train Epoch:

Train Epoch: 6 [0/60032 (0%)]	Loss: 0.105805
Train Epoch: 6 [1920/60032 (3%)]	Loss: 0.039563
Train Epoch: 6 [3840/60032 (6%)]	Loss: 0.028100
Train Epoch: 6 [5760/60032 (10%)]	Loss: 0.111757
Train Epoch: 6 [7680/60032 (13%)]	Loss: 0.012572
Train Epoch: 6 [9600/60032 (16%)]	Loss: 0.020595
Train Epoch: 6 [11520/60032 (19%)]	Loss: 0.007196
Train Epoch: 6 [13440/60032 (22%)]	Loss: 0.164978
Train Epoch: 6 [15360/60032 (26%)]	Loss: 0.030457
Train Epoch: 6 [17280/60032 (29%)]	Loss: 0.004209
Train Epoch: 6 [19200/60032 (32%)]	Loss: 0.082345
Train Epoch: 6 [21120/60032 (35%)]	Loss: 0.040032
Train Epoch: 6 [23040/60032 (38%)]	Loss: 0.041765
Train Epoch: 6 [24960/60032 (42%)]	Loss: 0.017844
Train Epoch: 6 [26880/60032 (45%)]	Loss: 0.010150
Train Epoch: 6 [28800/60032 (48%)]	Loss: 0.059283
Train Epoch: 6 [30720/60032 (51%)]	Loss: 0.032701
Train Epoch: 6 [32640/60032 (54%)]	Loss: 0.012541
Train Epoch: 6 [34560/60032 (58%)]	Loss: 0.044780
Train Epoch: 6 [36480/60032 (61%)]	Loss: 0.058152
Train Epoch:


Test set: Average loss: 0.0430, Accuracy: 9862/10000 (99%)

Wall time: 42min 58s
